Axon Area 
Markdown Overview 
1. Reading in Axonal Area of Single PNG file 
2. Reading in Directory of AxonAnalysis PNG and it sorts through them, puts it in an output folder and is used as inclusion criteria for analysis 
3. If Axon Area passes 800000 

In [3]:
''' First Cell: Single Axonal Area Calculation from PNG Image
    Input: PNG image file path - ENTER PNG FILE PATH after image_path =
    Output: Axon-covered area in µm²'''
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Load your PNG file
image_path = "/mnt/benshalom-nas/analysis/BatchTest/Media_Density_Experiment_T2_04142025_AR/250428/M08024/AxonTracking/000095/analysis/FootprintExtraction_v1/BA_12624403860676221272/Well3/FootprintNeuron#27.png"  # Replace with the actual path
img = Image.open(image_path).convert("RGB")

# Convert image to numpy array
img_np = np.array(img)

# Convert to grayscale using luminance formula
gray_img = np.dot(img_np[...,:3], [0.2989, 0.5870, 0.1140])

# Apply a grayscale threshold to isolate the signal (tweak this threshold as needed)
threshold = 30  # Adjust if needed
axon_mask = gray_img > threshold

# Estimate pixel size from scale bar: 100 µm ≈ 18 pixels → 1 pixel ≈ 5.56 µm
pixel_size_um = 100 / 18
pixel_area_um2 = pixel_size_um ** 2

# Calculate axonal area
axon_area_um2 = np.sum(axon_mask) * pixel_area_um2

if axon_area_um2 > 800000: 
    print("Detected Neuron")
else: 
    print("No Neuron Detected")
# Output results
print(f"Axon-covered area: {axon_area_um2:.2f} µm²")


No Neuron Detected
Axon-covered area: 765524.69 µm²


In [4]:
import os
import re
import shutil
from PIL import Image
import numpy as np
import csv
import pandas as pd

# CONFIGURATION
BASE_DIR = "/mnt/benshalom-nas/analysis/BatchTest/Media_Density_Experiment_T2_04142025_AR/"
OUTPUT_DIR = "/mnt/Vol20tb1/user_workspaces/shruti/analysis/AxonTracking_MDT2/"
OUTPUT_CSV = "axon_area_results.csv"
DETECTED_FOOTPRINTS_DIR = os.path.join(OUTPUT_DIR, "Detected_Footprints")

PIXELS_PER_100_UM = 18  # from scale bar
PIXEL_AREA_UM2 = (100 / PIXELS_PER_100_UM) ** 2  # µm² per pixel
THRESHOLD = 30  # threshold (adjust as needed)
DETECTION_THRESHOLD = 800000  # µm² threshold for neuron detection
FOLDER_SUFFIX = "272"  # Folder ending pattern (can be changed)

# Create output directories if they don't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DETECTED_FOOTPRINTS_DIR, exist_ok=True)

def is_target_image(filename):
    """Check if filename matches TrackingNeuron pattern"""
    return re.match(r"TrackingNeuron#\d+\.png", filename)

def extract_metadata(filepath):
    """
    Extract Date, Plate Number, AxonTracking ID, BA identifier, Analysis folder, Well number, Neuron number
    Example: .../250428/M08024/AxonTracking/000095/analysis/AxonAnalysis_v1/BA_12624403860676221272/Well3/TrackingNeuron#27.png
    """
    parts = filepath.split(os.sep)
    try:
        # Extract date (6-digit number like 250428)
        date = next((p for p in parts if re.match(r'^\d{6}$', p)), "Unknown")
        
        # Extract plate number (starts with M followed by digits)
        plate_num = next((p for p in parts if re.match(r'^M\d+', p)), "Unknown")
        
        # Extract tracking ID (comes after AxonTracking/)
        tracking_id = parts[parts.index("AxonTracking") + 1] if "AxonTracking" in parts else "Unknown"
        
        # Extract BA identifier (BA_xxxxx...)
        ba_match = next((p for p in parts if p.startswith("BA_")), None)
        ba_id = ba_match if ba_match else "Unknown"
        
        # Extract analysis folder ending in specified suffix (e.g., 272)
        analysis_folder = next((p for p in parts if p.endswith(FOLDER_SUFFIX)), "Unknown")
        
        # Extract well number
        well_match = re.search(r"Well(\d+)", filepath)
        well = f"Well{well_match.group(1)}" if well_match else "Unknown"
        
        # Extract neuron number
        neuron_match = re.search(r"TrackingNeuron#(\d+)", filepath)
        neuron = f"Neuron#{neuron_match.group(1)}" if neuron_match else "Unknown"
        
        return {
            "date": date,
            "plate_num": plate_num,
            "tracking_id": tracking_id,
            "ba_id": ba_id,
            "analysis_folder": analysis_folder,
            "well": well,
            "neuron": neuron
        }
    except Exception as e:
        print(f"Error extracting metadata from {filepath}: {e}")
        return {
            "date": "Unknown",
            "plate_num": "Unknown",
            "tracking_id": "Unknown",
            "ba_id": "Unknown",
            "analysis_folder": "Unknown",
            "well": "Unknown",
            "neuron": "Unknown"
        }

def analyze_image_grayscale(image_path, threshold=THRESHOLD):
    """
    Analyze image using grayscale conversion (luminance formula)
    """
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)
    
    # Convert to grayscale using luminance formula
    gray_img = np.dot(img_np[...,:3], [0.2989, 0.5870, 0.1140])
    
    # Apply threshold
    axon_mask = gray_img > threshold
    
    pixel_count = np.sum(axon_mask)
    area_um2 = pixel_count * PIXEL_AREA_UM2
    
    return area_um2, pixel_count

def analyze_image_red_channel(image_path, threshold=THRESHOLD):
    """
    Analyze image using red channel (for heatmap visualization)
    """
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)
    
    # Use red channel as signal indicator
    red_channel = img_np[:, :, 0]
    
    # Apply threshold
    signal_mask = red_channel > threshold
    
    pixel_count = np.sum(signal_mask)
    area_um2 = pixel_count * PIXEL_AREA_UM2
    
    return area_um2, pixel_count

def copy_detected_footprint(source_path, metadata):
    """
    Copy detected footprint to organized folder structure
    Creates: Detected_Footprints/Date/PlateNum/TrackingID/BA_ID/Well/TrackingNeuron#X.png
    """
    # Create nested folder structure
    dest_dir = os.path.join(
        DETECTED_FOOTPRINTS_DIR,
        metadata["date"],
        metadata["plate_num"],
        metadata["tracking_id"],
        metadata["ba_id"],
        metadata["well"]
    )
    os.makedirs(dest_dir, exist_ok=True)
    
    # Copy file
    filename = os.path.basename(source_path)
    dest_path = os.path.join(dest_dir, filename)
    shutil.copy2(source_path, dest_path)
    
    return dest_path

# Main processing
results = []
detected_count = 0
total_count = 0

print("🔍 Starting axon area analysis...")
print(f"Looking for TrackingNeuron#*.png files...")
print(f"Filtering for folders ending in '{FOLDER_SUFFIX}'...\n")
print("Testing both methods on first image to compare...\n")

first_image_tested = False

for root, _, files in os.walk(BASE_DIR):
    for file in files:
        if is_target_image(file):
            # Check if path contains folder ending with specified suffix
            if not any(part.endswith(FOLDER_SUFFIX) for part in root.split(os.sep)):
                continue
            
            total_count += 1
            full_path = os.path.join(root, file)
            metadata = extract_metadata(full_path)
            
            # Analyze image using grayscale method
            area_um2_gray, pixel_count_gray = analyze_image_grayscale(full_path)
            
            # Analyze image using red channel method
            area_um2_red, pixel_count_red = analyze_image_red_channel(full_path)
            
            # Test first image with both methods
            if not first_image_tested:
                print("="*80)
                print("COMPARISON TEST - First Image:")
                print(f"File: {os.path.basename(full_path)}")
                print(f"Path: {full_path}")
                print(f"Grayscale method: {area_um2_gray:.2f} µm² ({pixel_count_gray} pixels)")
                print(f"Red channel method: {area_um2_red:.2f} µm² ({pixel_count_red} pixels)")
                print("="*80 + "\n")
                first_image_tested = True
            
            # Use grayscale method (as specified in your original code)
            area_um2 = area_um2_gray
            pixel_count = pixel_count_gray
            
            # Determine if neuron is detected
            is_detected = area_um2 > DETECTION_THRESHOLD
            detection_status = "Detected Neuron" if is_detected else "No Neuron Detected"
            
            # Copy to Detected_Footprints if detected
            copied_path = None
            if is_detected:
                detected_count += 1
                copied_path = copy_detected_footprint(full_path, metadata)
                print(f"✅ {metadata['date']} | {metadata['plate_num']} | {metadata['ba_id']} | {metadata['well']} | {metadata['neuron']}: {area_um2:.2f} µm² - DETECTED")
            
            results.append({
                "Date": metadata["date"],
                "Plate Number": metadata["plate_num"],
                "Tracking ID": metadata["tracking_id"],
                "BA ID": metadata["ba_id"],
                "Analysis Folder": metadata["analysis_folder"],
                "Well": metadata["well"],
                "Neuron": metadata["neuron"],
                "Pixel Count": pixel_count,
                "Axon Area (µm²)": round(area_um2, 2),
                "Axon Area Red Channel (µm²)": round(area_um2_red, 2),  # For comparison
                "Detection Status": detection_status,
                "Threshold Used": THRESHOLD,
                "Detection Threshold (µm²)": DETECTION_THRESHOLD,
                "Copied to Detected Folder": "Yes" if copied_path else "No",
                "Original Path": full_path
            })

# Convert to DataFrame
df = pd.DataFrame(results)

# Sort by area (descending) to see largest footprints first
df = df.sort_values("Axon Area (µm²)", ascending=False)

# Save to CSV
csv_path = os.path.join(OUTPUT_DIR, OUTPUT_CSV)
df.to_csv(csv_path, index=False)

# Print summary
print("\n" + "="*80)
print("📊 ANALYSIS SUMMARY")
print("="*80)
print(f"Total images analyzed: {total_count}")
print(f"Neurons detected (>{DETECTION_THRESHOLD} µm²): {detected_count}")
print(f"Detection rate: {(detected_count/total_count*100):.1f}%" if total_count > 0 else "No images found")
print(f"\n✅ Results saved to: {csv_path}")
print(f"📁 Detected footprints copied to: {DETECTED_FOOTPRINTS_DIR}")
print("="*80)

# Preview top 10 results
print("\n🔬 Top 10 Largest Axon Areas:\n")
if not df.empty:
    print(df[["Date", "Plate Number", "BA ID", "Well", "Neuron", "Axon Area (µm²)", "Detection Status"]].head(10).to_string(index=False))
else:
    print("No images found to analyze.")

# Save detected-only CSV
detected_df = df[df["Detection Status"] == "Detected Neuron"]
if not detected_df.empty:
    detected_csv_path = os.path.join(OUTPUT_DIR, "detected_neurons_only.csv")
    detected_df.to_csv(detected_csv_path, index=False)
    print(f"\n✅ Detected neurons only saved to: {detected_csv_path}")
else:
    print(f"\n⚠️ No neurons detected above {DETECTION_THRESHOLD} µm² threshold")

print(f"\n💡 Note: Analysis filtered for folders ending in '{FOLDER_SUFFIX}'")
print(f"   To change this, modify FOLDER_SUFFIX variable at the top of the script")

🔍 Starting axon area analysis...
Looking for TrackingNeuron#*.png files...
Filtering for folders ending in '272'...

Testing both methods on first image to compare...

COMPARISON TEST - First Image:
File: TrackingNeuron#2.png
Path: /mnt/benshalom-nas/analysis/BatchTest/Media_Density_Experiment_T2_04142025_AR/250417/M08024/AxonTracking/000018/analysis/AxonAnalysis_v1/BA_12624403860676221272/Well1/TrackingNeuron#2.png
Grayscale method: 481234.57 µm² (15592 pixels)
Red channel method: 469382.72 µm² (15208 pixels)

✅ 250417 | M08024 | BA_12624403860676221272 | Well1 | Neuron#15: 832932.10 µm² - DETECTED
✅ 250417 | M08024 | BA_12624403860676221272 | Well2 | Neuron#20: 804197.53 µm² - DETECTED
✅ 250417 | M08024 | BA_12624403860676221272 | Well2 | Neuron#22: 802006.17 µm² - DETECTED
✅ 250417 | M08024 | BA_12624403860676221272 | Well4 | Neuron#28: 921481.48 µm² - DETECTED
✅ 250417 | M08024 | BA_12624403860676221272 | Well6 | Neuron#3: 917500.00 µm² - DETECTED
✅ 250417 | M08024 | BA_12624403860

In [ ]:
plates = sorted(df["Plate Number"].unique())
filtered = detected_df[detected_df["Plate Number"].isin(plates)]
counts = filtered.groupby(["Date", "Plate Number"]).size().unstack(fill_value=0).sort_index()
print(counts)

counts


Plate Number  M08024  M08035
Date                        
250417             6      10
250421             6      16
250424            15      54
250428            30      63
250501            76      62
250505            73      68
250508            52      35
250512           110      64
250515            56      23


Plate Number,M08024,M08035
Date,,
250417,6,10
250421,6,16
250424,15,54
250428,30,63
250501,76,62
250505,73,68
250508,52,35
250512,110,64
250515,56,23
